In [ ]:
import tsim
import numpy as np

The following circuit prepares the state
$$\frac{1}{2}\Big[(1 + e^{i\pi/4})|0\rangle + (1 - e^{i\pi/4})|1\rangle\Big]$$
and measures it in the Z basis.

In [ ]:
c = tsim.Circuit(
    """
    RX 0
    T 0
    H 0
    M 0
    """
)
c.diagram("timeline-svg", height=120)

We compile a sampler and generate 10 million samples:

In [ ]:
sampler = c.compile_sampler()

In [ ]:
samples = sampler.sample(shots=10_000_000, batch_size=1_000_000)
samples

The probability of measuring a 1 is ~14.6%, consistent with the expected value of
$\sin^2(\pi/8) \approx 0.146$.

In [ ]:
p1_noiseless = int(np.count_nonzero(samples)) / len(samples)
p1_noiseless

In [ ]:
print(np.sin(np.pi / 8) ** 2)

Let's now encode the above circuit using the [7,1,3] Steane code, and include some depolarizing noise:

In [ ]:
p = 0.001
circuit = tsim.Circuit(
    f"""
    RX 6
    T 6
    H 6
    R 0 1 2 3 4 5
    TICK
    SQRT_Y_DAG 0 1 2 3 4 5
    DEPOLARIZE1({p}) 0 1 2 3 4 5
    TICK
    CZ 1 2 3 4 5 6
    DEPOLARIZE2({p}) 1 2 3 4
    TICK
    SQRT_Y 6
    DEPOLARIZE1({p}) 6
    TICK
    CZ 0 3 2 5 4 6
    DEPOLARIZE2({p}) 0 3 2 5 4 6
    TICK
    SQRT_Y 2 3 4 5 6
    DEPOLARIZE1({p}) 2 3 4 5 6
    TICK
    CZ 0 1 2 3 4 5
    DEPOLARIZE2({p}) 0 1 2 3 4 5
    TICK
    DEPOLARIZE1({p}) 0 1 2 3 4 5 6
    SQRT_Y 1 2 4
    X 3
    TICK
    M 0 1 2 3 4 5 6
    DETECTOR rec[-7] rec[-6] rec[-5] rec[-4]
    DETECTOR rec[-6] rec[-5] rec[-3] rec[-2]
    DETECTOR rec[-5] rec[-4] rec[-3] rec[-1]
    OBSERVABLE_INCLUDE(0) rec[-7] rec[-6] rec[-2]
    """
)
c.diagram("timeline-svg", height=450)

In [ ]:
circuit = tsim.Circuit(
    """
    R 0 1 2 3 4 5
    RX 6
    T 6
    DEPOLARIZE1(0.001) 0 1 2 3 4 5
    ...
    """
)

In [ ]:
import pyzx as zx
from tsim.graph_util import transform_error_basis, squash_graph

g = c.get_sampling_graph(sample_detectors=True)
zx.full_reduce(g)
squash_graph(g)
g, _ = transform_error_basis(g)
zx.draw(g)

We can sample detectors and observables. Detectors are the syndrome bits that indicate the presence of errors when they are 1. Since we have noise in our simulation, we expect some of them to be 1.

In [ ]:
det_sampler = circuit.compile_detector_sampler()
det_samples, obs_samples = det_sampler.sample(shots=100_000, separate_observables=True)

In [ ]:
det_samples

The observable corresponds to the logical bit. Since the [7,1,3] Steane code has only a single logical qubit, we have a single observable. In the absence of any noise, the statistics of the logical observable should match the statistics of the measurements of our single-qubit circuit from the very beginning of this tutorial. But since the circuit is noisy, the probability of measuring 1 is slightly increased:

In [ ]:
obs_samples

In [ ]:
int(np.count_nonzero(obs_samples)) / len(obs_samples)

We can fix the statistics by performing error detection. Here, we simply discard all shots that don't have perfect stabilizers. This brings the probability back to 14.6%, close to the ideal value:

In [ ]:
perfect_stabilizers = np.all(det_samples == 0, axis=1)
post_selected_obs = obs_samples[perfect_stabilizers]
int(np.count_nonzero(post_selected_obs)) / len(post_selected_obs)

### Detector Error Model

Instead of simply discarding shots, we can decode the data. For this, we need to define a detector error model. With `tsim` (or `stim`), this is straightforward:

In [ ]:
dem = c.detector_error_model()
dem

This is an error model in `stim.DetectorErrorModel` format. Each line corresponds to an error mechanism with a certain probability and a signature, i.e., which detectors and observables are flipped by the error mechanism.

An equivalent representation of the error model is in terms of a check matrix, observable matrix, and prior probabilities:

In [ ]:
from beliefmatching import detector_error_model_to_check_matrices

cm = detector_error_model_to_check_matrices(dem, allow_undecomposed_hyperedges=True)

check_matrix = cm.check_matrix
observable_matrix = cm.observables_matrix
priors = cm.priors

In [ ]:
check_matrix.toarray()

In [ ]:
observable_matrix.toarray()

In [ ]:
priors

### Decoding

We can use open-source decoders to decode our data. Here, we use the `tesseract` decoder and Belief Propagation decoder from the `ldpc` package. Decoders must be initialized with the detector error model.

Then, the decoder will take the detector/syndrome data, and return a recommendation of whether to flip our observable bit. After performing that flip, the statistics of the corrected observable show a 14.7% probability of measuring a 1, close to the ideal value of 14.6% (but not quite as good as the post-selection method).

Generally, post-selection performs better since it can correct errors of weight `d-1`, whereas decoders can only correct errors of weight up to `(d-1)/2`.

In [ ]:
from tesseract_decoder import tesseract

config = tesseract.TesseractConfig(dem=c.detector_error_model())
decoder = config.compile_decoder()


obs_corrected = np.zeros_like(obs_samples)
for i, det_sample in enumerate(det_samples):
    flip_obs = decoder.decode(det_sample)
    obs_corrected[i] = np.logical_xor(obs_samples[i], flip_obs[0])

print("Uncorrected:", int(np.count_nonzero(obs_samples)) / len(obs_samples))
print("Corrected:  ", int(np.count_nonzero(obs_corrected)) / len(obs_corrected))
print("Noiseless:  ", p1_noiseless)

Let's also use a different decoder: Belief Propagation from the `ldpc` package. This decoder needs to be initialized with the check matrix and prior probabilities. Additionally, it returns estimated error mechanisms, from which we compute the estimated observable flip using `(estimated_error_mechanisms @ observable_matrix.T) % 2`.

In [ ]:
from ldpc.bp_decoder import BpDecoder

decoder = BpDecoder(
    pcm=check_matrix,
    error_channel=priors,
    bp_method="minimum_sum",
    ms_scaling_factor=0.5,
)


obs_corrected = np.zeros_like(obs_samples)
for i, det_sample in enumerate(det_samples):
    estimated_error_mechanisms = decoder.decode(det_sample)
    flip_obs = (estimated_error_mechanisms @ observable_matrix.T) % 2
    obs_corrected[i] = np.logical_xor(obs_samples[i], flip_obs[0])

print("Uncorrected:", int(np.count_nonzero(obs_samples)) / len(obs_samples))
print("Corrected:  ", int(np.count_nonzero(obs_corrected)) / len(obs_corrected))
print("Noiseless:  ", p1_noiseless)

### Converting measurement bits to detector and observable bits

Detectors and observable bits are just XORs of measurement bits. We can use the `m2d_converter` to convert measurement bits to detector and observable bits.

In [ ]:
m2d_converter = c._stim_circ.compile_m2d_converter()

In [ ]:
sampler = c.compile_sampler()
samples = sampler.sample(shots=10)
samples

In [ ]:
det_samples, obs_samples = m2d_converter.convert(
    measurements=samples, separate_observables=True
)
print("Detection events:\n", det_samples)
print("Logical bit:\n", obs_samples)